# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119393e+02     1.387779e+00
 * time: 0.662071943283081
     1     1.024662e+01     8.473830e-01
 * time: 1.688188076019287
     2    -1.212260e+01     9.133589e-01
 * time: 2.210616111755371
     3    -3.467211e+01     6.674958e-01
 * time: 2.9383349418640137
     4    -4.831907e+01     5.177543e-01
 * time: 3.6729719638824463
     5    -5.730133e+01     2.025913e-01
 * time: 4.416685104370117
     6    -6.002337e+01     1.441489e-01
 * time: 4.899590969085693
     7    -6.102940e+01     4.511635e-02
 * time: 5.382759094238281
     8    -6.139173e+01     7.042536e-02
 * time: 5.8663880825042725
     9    -6.163669e+01     3.785200e-02
 * time: 6.367999076843262
    10    -6.180702e+01     2.703276e-02
 * time: 6.852473974227905
    11    -6.195974e+01     2.058759e-02
 * time: 7.336190938949585
    12    -6.202529e+01     1.806328e-02
 * time: 7.820631980895996
    13    -6.209263e+01     1.340744e-02
 * time: 8.304433107376099
    

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671073
    AtomicLocal         -18.8557668
    AtomicNonlocal      14.8522629
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336822

    total               -62.261666456831